# SOIL DATA - Troubles finding what to use?!

- SSURGO is more detail can I can work with in the timeframe I need it
    - USDA NRCS 
        - https://sdmdataaccess.nrcs.usda.gov/Default.aspx
- STATSGO may be too limiting
    - USDA NRCS (shares tables in data source as above)
- How about The Global Soil Dataset for Earth System Modeling
    - Land-Atmosphere Interaction Research Group at Sun Yat-sen University
        - http://globalchange.bnu.edu.cn/research/soilwd.jsp

## Explore Soil Organic Carbon Density in The Global Soil Dataset

_I've already done quite a bit of noodling around the USDA data in other notebooks so I'll take a look at whether this one will fit my needs better (and my timeframe and level of pre-aggregation and simplification desired)._

### Load the NetCDF 

Network common data form (NetCDF) is commonly used to store multidimensional geographic data, and especially common with geographic time series data. I'll load the 5 minute geospatial resolution version of the Soil organic carbon density (SOCD5min.zip) NetCDF file in after downloading it from The Global Soil Dataset.

In [ ]:
# configure Google geocoding API key constant/environment variable for work with cesiumpy?

In [ ]:
# if xarray is not yet installed, uncomment and run one of the following lines (either/or), 
# which only need to be run once
# !pip install xarray
# I probably should have used conda because my virtual envelope is maintained with conda, so if I run into problems I will uninstall with pip and reinstall with conda
# !conda install xarray 

In [ ]:
# if folium is not yet installed, uncomment and run one of the following lines (either/or), 
# which only need to be run once
# !conda install --yes folium -c conda-forge
# !pip install --yes folium

In [ ]:
# view plots inside the notebook
%matplotlib inline  
# import package dependencies for environment
import netCDF4 as nc
import xarray as xr
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import geopandas
# import fiona to view a full list of supported formats for geopandas.GeoDataFrame.to_file() method
import fiona; fiona.supported_drivers
# import cesiumpy # commented out due to unresolved AttributeError: partially initialized module 'cesiumpy' has no attribute 'data' (most likely due to a circular import)

In [ ]:
# import more packages for additional plotting tests
import geoplot as gplt
import geoplot.crs as gcrs
import imageio
import pathlib
import mapclassify as mc
from shapely import wkt

In [ ]:
# check working directory using Shell command in IPython syntax preceded by '!'
!pwd

In [ ]:
# list directory contents
!ls

In [ ]:
# can I see my data folder in the root directory of my project 
# (i.e. in the parent of current analysis/notebooks folder working directory)?
#!echo ../*/ #alternately
!ls ..

In [ ]:
# now can I see the files in my data folder?
!ls ../data

In [ ]:
# Great! Now I've checked and copied the filename from right here in my Notebook!
# load NetCDF .nc file using the netcdf4 package (note can also be done using gdal package)
fn = '../data/SOCD5min.nc' # relative path to netcdf file
ds = nc.Dataset(fn) # read as netcdf dataset
# view info about the variables
print(ds)
# print(ds.__dict__) #alternately print metadata as a Python dictionary

In [ ]:
# access information about the single specific variable metadata (that is not a dimension) 
# SOCD is Soil Organic Carbon Density
# measured and recorded in t/ha (tonnes per hectare)
print(ds['SOCD'])

In [ ]:
# just print dimensions as a python dictionary
print(ds.dimensions)

In [ ]:
# access the data values just like a numpy array
socd = ds['SOCD'][:]
print(socd)

In [ ]:
socd

### Need to get this in a more workable format

I'll try to understand this data format more.
Ultimately I want to transform it into a Pandas dataframe.

In [ ]:
# check the type of the new named variable socd
type(socd)

In [ ]:
# see the shape of the array
socd.shape

In [ ]:
# view an element about midway through
socd[0, 1000, 1000]

In [ ]:
# try another
socd[0, 0, 0]

In [ ]:
# get the non-masked data, specifically by removing rows with all masked data
# returns invlid syntax error on the axis=1
# socd_unmasked_all = socd[~socd.mask.all[axis=1]]

In [ ]:
# the compressed method will remove masked items, but flattens the result to a 1 dimensional array
# so I've lost the location dimensions that way
socd_compressed = socd.compressed()
print(socd_compressed)
socd_compressed.shape

In [ ]:
# reshape the masked array to 2D, to try to make it into a dataframe
socd.reshape(-1, 1)

### Halp!! 
Here's the point where I asked for help. Xarray to the rescue!
Thank you Dr. Larry Gray for your consultation that led me to this pivot!

In [ ]:
# use the xarray package instead of netCDF4 to view and process the dataset from here
# added to packages import list at top of notebook
# read the data file in with xarray instead and assign it to ds variable
ds = xr.open_dataset("../data/SOCD5min.nc")
# transform it to a dataframe assigned to df variable
df = ds.to_dataframe()

### Yay, no more errors!

Now this is what I'm used to data looking like!

In [ ]:
# view the dataframe
df

In [ ]:
# take a look at the SOCD column
# to better understand the index structure
df.SOCD

In [ ]:
# view all the column variable names in dataframe
df.columns

In [ ]:
# view the index
df.index

In [ ]:
# view the unique set of values possible for depth (in centimeters per documentation; 8 levels recorded)
depths = df['depth'].unique()
depths

In [ ]:
# store the depths variable for use in other notebooks
%store depths

In [ ]:
# # commented out because it was unnecessary to remove index in order to remove the na values, and
# # keep for reference in case I need a different structure to plot something with
# # reset the index of the dataframe to remove the hierarchical index structure
# df3 = df.reset_index()
# # subset by variable name SOCD only the records with SOCD values is na is FALSE, using '~'
# df3[~df3.SOCD.isna()]

In [ ]:
# try the subset to remove na values without going through the flattening of the index first
# it works without resetting the index
df2 = df[~df.SOCD.isna()]

In [ ]:
# view the dataframe now, which retains its original indexing, but has only records with values
df2

In [ ]:
# what's the maximum value that appears for SOCD for any depth
df2['SOCD'].max()

In [ ]:
# what's the minimum value that appears for SOCD for any depth
df2['SOCD'].min()

In [ ]:
# see mean for depth
df2['SOCD'].mean()

In [ ]:
# see median for depth
df2['SOCD'].median()

In [ ]:
# take aggregates of SOCD across all depths, for each location, i.e. when 
# grouped by the first and second level of the index (i.e. by lon then lat)... 
# ...is there any relevant aggregate for SOCD which is listed as a % in the documentation? Sum does not seem accurate
# skip for now
# df2.groupby(level=[0,1]).aggregate()

In [ ]:
# flatten the index of the overall df2 dataframe (with NaN removed, before summary)
df2flat = df2.reset_index()
df2flat

In [ ]:
# check max SOCD again now
# what's the maximum value that appears for SOCD for any depth
df2flat['SOCD'].max()

In [ ]:
# export the first 50 rows of the dataframe for instructor/cohort review
df2flat.head(50).to_csv("../data/SOCD5min_missingvaluesremoved_excerpt.csv")

In [ ]:
# check how much disk use space is in use in data folder, for github large file storage (lfs) considerations
# "*.csv" and "*.nc" are beign logged in git lfs
!du -sh ../data

## For mapping, translate to spatial geometry

I'll use the GeoPandas package to jump from longitude and latitude columns into a mappable format. In order to try to stay true to the original raw dataset as much as possible, I'll test on the dataframe that includes NA values first to see how GeoPandas handles and displays NA values.

In [ ]:
# first try this with NA values left in, to stay true to data source as much as possible, and
# first flatten the multiple indexes and reset the index, so that lon, lat will be recognized at keys
df = df.reset_index()

In [ ]:
# check this dataframe's columns; lon, lat, and depth have been added to columns with SOCD now
df.columns

In [ ]:
# view just the first 24 records
df.head(24)

In [ ]:
# view just the last 10 records
df.tail(16)

In [ ]:
# view the new index; it's now a sequential index
df.index

In [ ]:
# translate lon and lat columns into a spatial geometry variable in a GeoPandas dataframe
# commented out after initial test run successfully, but as too intensive i.e. slow to run
# gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))

In [ ]:
# for the dataframe with NA removed
# translate lon and lat columns into a spatial geometry variable in a GeoPandas dataframe
gdf2flat = geopandas.GeoDataFrame(df2flat, geometry=geopandas.points_from_xy(df2flat.lon, df2flat.lat))

In [ ]:
# view the gdf2flat GeoPandas dataframe with geometry column now
gdf2flat

In [ ]:
# translate lon and lat columns into a spatial geometry variable in a GeoPandas dataframe
gdf2flat = geopandas.GeoDataFrame(df2flat, geometry=geopandas.points_from_xy(df2flat.lon, df2flat.lat))

In [ ]:
# view the gdf2sum GeoPandas dataframe with geometry column now
gdf2flat

In [ ]:
#check for NaN values in geometry
print(gdf2flat['geometry'].isnull().values.any())
#check for NaN values in entire dataframe
print(gdf2flat['geometry'].isnull().values.all())
#count NaN values in geometry
print(gdf2flat.isnull().values.any())
#count NaN values in entire dataframe
print(gdf2flat.isnull().sum().sum)

In [ ]:
# view the data types
print(gdf2flat['geometry'].dtype)
print(gdf2flat.dtypes)

### Mapping and plotting

In [ ]:
# making maps with GeoPandas, look at the geometry points for all SOCD
# using standard 'pyplot' line style options
gdf2flat.plot(marker='*', color='#9b7653', markersize=5); # named 'dirt' color code just looks better than 'brown'

In [ ]:
# check if gdf2flat has a CRS
gdf2flat.crs is None

In [ ]:
# Set a CRS on the geo dataframe object first
# according to data source documentation, the coordinate system is WGS_1984
# readme file is available at http://globalchange.bnu.edu.cn/download/doc/worldsoil/readme.zip
gdf2flat.crs = "EPSG:4326"

In [ ]:
# check the crs now
gdf2flat.crs

In [ ]:
# load the built in world natural earth low resolution dataset
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

#check if it has a CRS
print(world.crs is None)
world.crs

In [ ]:
# check CRS --Before combining maps, however, remember to always ensure they share a common CRS (so they will align).
# no need to convert because they match
# gdf2flat = gdf2flat.to_crs(world.crs)

In [ ]:
# plot points on the built in world map
base = world.plot(color='white', edgecolor='black')
gdf2flat.plot(ax=base, marker='o', color='#9b7653', markersize=1); # named 'dirt' color code just looks better than 'brown'

### Export data to a Spatial file format

Using Geopandas I will export to a GeoJSON file to upload data to work in Cesium ion/Cesium Story

In [ ]:
# export gdf2flat to GeoJSON
# gdf2flat.to_file("../data/gdf2flat.geojson", driver='GeoJSON')

### More mapping trials

In [ ]:
# using geoplot package to plot the points on the same world map
ax = gplt.polyplot(world, edgecolor='black')
gplt.pointplot(gdf2flat, ax=ax, marker='o', color='#9b7653')

In [ ]:
# check geometry of base and its coordinate reference system (crs) (...and possibly did this earlier already)
print(world.geometry)
print(world.crs)

In [ ]:
# try to explicitly set the geometry even though it appears to already be there
world = world.set_geometry("geometry")
world.geometry

In [ ]:
# check geometry of gdf2flat and its crs also
print(gdf2flat.geometry)
print(gdf2flat.crs)

In [ ]:
# try to explicitly set the geometry of gdf2flat even though it appears to already be there
gdf2flat = gdf2flat.set_geometry("geometry")
gdf2flat.geometry

In [ ]:
# using shapely import wkt convert the column geometry to actual geometries 
# (I think it was already reading as geometry before this, but this is a test of a recommended debugging solution for "ValueError: The input GeoDataFrame does not have a "geometry" column set.")
# commented out test as failed (which is good actually) with "TypeError: Only str is accepted." indicating 
# my geometry was not strings but actually already geometry
# gdf2flat['geometry'] = gdf2flat['geometry'].apply(wkt.loads)

In [ ]:
# loop through dataframe to validate for any invalid geometries
for index, row in gdf2flat.iterrows():
    geom = row['geometry']
    if len(geom.coords) != 1:
          print("This row has an invalid point geometry")
          # this is just one example of invalid geometries, there may be others, ...

In [ ]:
# check geometry as a GeoSeries for missing values
gdf2flat["geometry"].isna()

In [ ]:
# check geometry as a GeoSeries for empty geometries
gdf2flat["geometry"].is_empty

In [ ]:
# collect only those where FALSE (i.e. only valid instances) in either case (or |) using a mask
gdf2flat_invalids = gdf2flat["geometry"][~(gdf2flat["geometry"].is_empty | gdf2flat["geometry"].isna())]
gdf2flat_invalids # matching length of gdf2flat confirms no invalid entries were found and removed       

In [ ]:
# use magic store command to cache variable to read it from other Jupyter Notebooks
# to read it into another notebook, use %store -r gdf2flat
%store gdf2flat